In [2]:
from transformers import ViTFeatureExtractor, ViTForImageClassification, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict, load_metric
import numpy as np
import torch
from PIL import Image

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load and preprocess dataset
ds = load_dataset('betul2').rename_column('label', 'labels')
ds = ds['train'].train_test_split(test_size=0.1)
train_val = ds['train'].train_test_split(test_size=0.1)
ds['train'], ds['validation'], ds['test'] = train_val['train'], train_val['test'], ds['test']
dataset = DatasetDict(ds)

# Initialize feature extractor and model
model_name_or_path = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)
labels = ds['train'].features['labels'].names
model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
).to(device)

# Transform function
def transform(example_batch):
    images = [np.moveaxis(np.array(x.convert('RGB')), -1, 0) for x in example_batch['image']]
    inputs = feature_extractor(images, return_tensors='pt')
    inputs['labels'] = example_batch['labels']
    return inputs

# Prepare dataset
prepared_ds = ds.with_transform(transform)

# Data collator
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

# Metric and compute function
metric = load_metric("accuracy", trust_remote_code=True)
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

# Training arguments
training_args = TrainingArguments(
    output_dir="./vit-base--v5",
    per_device_train_batch_size=16,
    evaluation_strategy="steps",
    num_train_epochs=4,
    fp16=True,
    save_steps=100,
    eval_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to='tensorboard',
    load_best_model_at_end=True,
)

# Trainer initialization
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["validation"],
    tokenizer=feature_extractor,
)

# Train and evaluate
train_results = trainer.train()
trainer.save_model()
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

metrics = trainer.evaluate(prepared_ds['test'])
trainer.save_metrics("eval", metrics)

# Model card creation
kwargs = {
    "finetuned_from": model.config._name_or_path,
    "tasks": "image-classification",
    "dataset": 'custom brats layers',
    "tags": ['image-classification'],
}
if training_args.push_to_hub:
    trainer.push_to_hub('🍻 cheers', **kwargs)
else:
    trainer.create_model_card(**kwargs)


Resolving data files:   0%|          | 0/3662 [00:00<?, ?it/s]

C:\Users\semih\AppData\Roaming\Python\Python312\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\semih\AppData\Roaming\Python\Python312\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/744 [00:00<?, ?it/s]

C:\Users\semih\AppData\Roaming\Python\Python312\site-packages\transformers\models\vit\modeling_vit.py:252: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  context_layer = torch.nn.functional.scaled_dot_product_attention(


{'loss': 1.1698, 'grad_norm': 1.4556313753128052, 'learning_rate': 0.00019731182795698925, 'epoch': 0.05}
{'loss': 0.852, 'grad_norm': 1.0453912019729614, 'learning_rate': 0.0001946236559139785, 'epoch': 0.11}
{'loss': 0.8861, 'grad_norm': 0.5025181770324707, 'learning_rate': 0.00019193548387096775, 'epoch': 0.16}
{'loss': 0.7948, 'grad_norm': 10.801403045654297, 'learning_rate': 0.000189247311827957, 'epoch': 0.22}
{'loss': 0.6557, 'grad_norm': 2.3779098987579346, 'learning_rate': 0.00018655913978494625, 'epoch': 0.27}
{'loss': 0.8153, 'grad_norm': 2.8715460300445557, 'learning_rate': 0.00018387096774193548, 'epoch': 0.32}
{'loss': 0.8188, 'grad_norm': 0.9872220158576965, 'learning_rate': 0.00018118279569892475, 'epoch': 0.38}
{'loss': 0.5689, 'grad_norm': 1.0359077453613281, 'learning_rate': 0.00017849462365591398, 'epoch': 0.43}
{'loss': 0.7094, 'grad_norm': 1.143519639968872, 'learning_rate': 0.00017580645161290325, 'epoch': 0.48}
{'loss': 0.8534, 'grad_norm': 1.0831692218780518, '

...

  0%|          | 0/42 [00:00<?, ?it/s]

{'eval_loss': 0.5549391508102417, 'eval_accuracy': 0.796969696969697, 'eval_runtime': 2.0841, 'eval_samples_per_second': 158.339, 'eval_steps_per_second': 20.152, 'epoch': 3.23}
{'loss': 0.3444, 'grad_norm': 6.237390995025635, 'learning_rate': 3.602150537634409e-05, 'epoch': 3.28}
{'loss': 0.3708, 'grad_norm': 5.466879367828369, 'learning_rate': 3.3333333333333335e-05, 'epoch': 3.33}
{'loss': 0.2087, 'grad_norm': 0.4771074950695038, 'learning_rate': 3.0645161290322585e-05, 'epoch': 3.39}
{'loss': 0.3231, 'grad_norm': 1.6142545938491821, 'learning_rate': 2.7956989247311828e-05, 'epoch': 3.44}
{'loss': 0.3607, 'grad_norm': 2.0825023651123047, 'learning_rate': 2.5268817204301075e-05, 'epoch': 3.49}
{'loss': 0.2245, 'grad_norm': 2.978687286376953, 'learning_rate': 2.258064516129032e-05, 'epoch': 3.55}
{'loss': 0.3053, 'grad_norm': 1.6812710762023926, 'learning_rate': 1.989247311827957e-05, 'epoch': 3.6}
{'loss': 0.2696, 'grad_norm': 3.742980480194092, 'learning_rate': 1.7204301075268818e-0

  0%|          | 0/42 [00:00<?, ?it/s]

{'eval_loss': 0.5628989338874817, 'eval_accuracy': 0.7818181818181819, 'eval_runtime': 1.9213, 'eval_samples_per_second': 171.759, 'eval_steps_per_second': 21.86, 'epoch': 3.76}
{'loss': 0.3178, 'grad_norm': 0.31403791904449463, 'learning_rate': 9.13978494623656e-06, 'epoch': 3.82}
{'loss': 0.2307, 'grad_norm': 2.097884178161621, 'learning_rate': 6.451612903225806e-06, 'epoch': 3.87}
{'loss': 0.3203, 'grad_norm': 3.1524269580841064, 'learning_rate': 3.763440860215054e-06, 'epoch': 3.92}
{'loss': 0.3157, 'grad_norm': 1.4197919368743896, 'learning_rate': 1.0752688172043011e-06, 'epoch': 3.98}
{'train_runtime': 164.9859, 'train_samples_per_second': 71.885, 'train_steps_per_second': 4.509, 'train_loss': 0.5454547696857042, 'epoch': 4.0}


  0%|          | 0/46 [00:00<?, ?it/s]

In [3]:
trainer.evaluate(prepared_ds['test'])

  0%|          | 0/46 [00:00<?, ?it/s]

{'eval_loss': 0.4978531301021576,
 'eval_accuracy': 0.8256130790190735,
 'eval_runtime': 2.3023,
 'eval_samples_per_second': 159.403,
 'eval_steps_per_second': 19.98,
 'epoch': 4.0}

In [4]:
print(metrics)

{'eval_loss': 0.4978531301021576, 'eval_accuracy': 0.8256130790190735, 'eval_runtime': 2.2373, 'eval_samples_per_second': 164.039, 'eval_steps_per_second': 20.561, 'epoch': 4.0}
